# [Module 2.1] 모델 준비, AWS IoT Thing, Device Fleet 생성 및 엣지 디바이스 등록

이 노트북은 아래와 같은 주요 작업을 합니다.
### 1. 모델 준비

- mobile-net pre-trained model다운로드
- Neo로  모델 컴파일
- 컴파일된 모델을 패키징

### 2. AWS IoT Thing 생성
##### 엣지 디바이스 --> AWS IoT thing 인증 수행 --> SageMaker 엣지 매니저 엔드포인트 호출
SageMaker 엣지 매니저는 AWS IoT 코어를 사용하여 디바이스를 인증하므로 AWS 클라우드의 SageMaker 엣지 매니저 엔드포인트를 호출할 수 있습니다. 

- IoT Thing Name, Iot Thing Type을 생성함

### 3. Device Fleet 생성
    - Fleet 이 사용할 Role 생성 및 정책 추가 (예: SageMaker2Fleet2gsmoon)
    - Role 에 trust relationship 추가

### 4. 엣지 디바이스 device fleet에 등록
- device_name, IotThingName 인자로 전달

### 5. AWS IoT Thing 에 Client Certificate을 생성 및 등록
- private key, public key, X.509 certificateㅇ르 생성
- AWS IoT 에 certificate을 등록 및 활성화
    - role_alias_name을 가지고 role_alias를 가져오기
        - role_alias_name: create_device_fleet() 호출시에 자동 생성
    - certificate에 권한 할당할 policy 생성 및 추가
    - credential provider 를 위한 엔드포인트 생성
        -  보안 토큰에 대한 요청을 인증하기 위해 사용됨
- Amazon Root CA file을 얻고, S3 에 업로드
- 엔드포인트를 사용하여 보안 토큰을 반환하도록 자격 증명 공급자에게 HTTPS 요청을 보냅니다.otCA1.pem $endpoint
- ```AmazonRootCA1.pem, iot.pem.crt, iot_key.pem.key``` s3에 업로드 (IoT Certificate 라 지칭)
    * SageMaker Edge Manager Agent 를 설정시에 사용함



In [2]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

role = get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

In [3]:
folder = 'DEMO-Sandbox-Sagemaker-Edge'
compilation_output_sub_folder = folder + '/compilation-output'
iot_folder = folder + '/iot'

# S3 Location to save the model artifact after compilation
s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)

## SageMaker Neo를 사용하여 Model 컴파일

Sagemaker client 생성

In [4]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

### pretrained Keras model 다운로드

In [5]:
import tensorflow as tf

model = tf.keras.applications.MobileNetV2()
model.save('mobilenet_v2.h5')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
14540800/14536120 [==============================] - 0s 0us/step


### 모델 tar 파일 압축 및 S3 업로드

In [6]:
import tarfile

with tarfile.open('mobilenet_v2.tar.gz', mode='w:gz') as archive:
    archive.add('mobilenet_v2.h5')

In [8]:
keras_model_path = sess.upload_data('mobilenet_v2.tar.gz', bucket, folder)
print("keras_model_path: ", keras_model_path)

keras_model_path:  s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/mobilenet_v2.tar.gz


### 모델 Neo로 컴파일
**[중요]**: 
- ```create_compilation_job ()```를 호출 할 때 사용자는 성공적인 컴파일을 위해 모델에 필요한 모든 올바른 입력 모양을 제공해야합니다.
- 다른 모델을 사용하는 경우 **프레임 워크와 입력 데이터 형태(Shape)**을 올바르게 지정해야합니다.

In [9]:
keras_model_data_shape = '{"input_1":[1,3,224,224]}'
keras_model_framework = 'keras'
target_device = 'ml_c5'

In [10]:
import time
keras_compilation_job_name = 'Sagemaker-Edge-'+ str(time.time()).split('.')[0]
print('Compilation job for %s started' % keras_compilation_job_name)

response = sagemaker_client.create_compilation_job(
        CompilationJobName=keras_compilation_job_name,
        RoleArn=role,
        InputConfig={
            'S3Uri': keras_model_path,
            'DataInputConfig': keras_model_data_shape,
            'Framework': keras_model_framework.upper()
        },
        OutputConfig={
            'S3OutputLocation': s3_compilation_output_location,
            'TargetDevice': target_device 
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 900
        }
    )

print(response)

# Poll every 30 sec
while True:
    response = sagemaker_client.describe_compilation_job(CompilationJobName=keras_compilation_job_name)
    if response['CompilationJobStatus'] == 'COMPLETED':
        break
    elif response['CompilationJobStatus'] == 'FAILED':
        raise RuntimeError('Compilation failed')
    print('Compiling ...')
    time.sleep(30)
print('Done!')

Compilation job for Sagemaker-Edge-1616309476 started
{'CompilationJobArn': 'arn:aws:sagemaker:us-east-2:057716757052:compilation-job/Sagemaker-Edge-1616309476', 'ResponseMetadata': {'RequestId': 'd570cf12-2d52-4712-acde-55f8d626d98f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd570cf12-2d52-4712-acde-55f8d626d98f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '106', 'date': 'Sun, 21 Mar 2021 06:51:16 GMT'}, 'RetryAttempts': 0}}
Compiling ...
Compiling ...
Compiling ...
Done!


## Keras Model 패키징

In [11]:
keras_packaged_model_name = "keras-model"
keras_model_version = "1.0"
keras_model_package = '{}-{}.tar.gz'.format(keras_packaged_model_name, keras_model_version)

In [12]:
keras_packaging_job_name=keras_compilation_job_name+"-packaging"
response = sagemaker_client.create_edge_packaging_job(
    RoleArn=role,
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location,
    },
    ModelName=keras_packaged_model_name,
    ModelVersion=keras_model_version,
    EdgePackagingJobName=keras_packaging_job_name,
    CompilationJobName=keras_compilation_job_name,
)

print(response)

# Poll every 30 sec
while True:
    job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=keras_packaging_job_name)
    if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
        break
    elif job_status['EdgePackagingJobStatus'] == 'FAILED':
        raise RuntimeError('Edge Packaging failed')
    print('Packaging ...')
    time.sleep(30)
print('Done!')

{'ResponseMetadata': {'RequestId': 'eb4afdc4-aa9b-42d9-9df7-0e91c2cbeb5b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'eb4afdc4-aa9b-42d9-9df7-0e91c2cbeb5b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '0', 'date': 'Sun, 21 Mar 2021 06:52:55 GMT'}, 'RetryAttempts': 0}}
Packaging ...
Done!


In [14]:
keras_model_data = job_status["ModelArtifact"]

In [17]:
print("keras_packaged_model_data: ", keras_model_data)

keras_packaged_model_data:  s3://sagemaker-us-east-2-057716757052/DEMO-Sandbox-Sagemaker-Edge/compilation-output/keras-model-1.0.tar.gz


## AWS IoT thing 생성

##### 엣지 디바이스 --> AWS IoT thing 인증 수행 --> SageMaker 엣지 매니저 엔드포인트 호출

SageMaker 엣지 매니저는 AWS IoT 코어를 사용하여 디바이스를 인증하므로 AWS 클라우드의 SageMaker 엣지 매니저 엔드포인트를 호출할 수 있습니다. 

엣지 디바이스에서 AWS 서비스를 사용하려면 먼저 인증해야 합니다.AWS IoT 기반 인증을 통해 이 작업을 수행하는 것이 좋습니다. 자세한 내용은 [여기](https://docs.aws.amazon.com/iot/latest/developerguide/authorizing-direct-aws.html) 및 [여기](https://aws.amazon.com/blogs/security/how-to-eliminate-the-need-for-hardcoded-aws-credentials-in-devices-by-using-the-aws-iot-credentials-provider/).




In [18]:
iot_client = boto3.client('iot', region_name=region)

In [19]:
iot_thing_name = 'sagemaker-edge-thing-demo-sandbox'
iot_thing_type = 'SagemakerEdgeDemoSandBox'

In [20]:
iot_client.create_thing_type(
    thingTypeName=iot_thing_type
)

{'ResponseMetadata': {'RequestId': '3209e83d-c958-4765-879f-dd1485e0f644',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 21 Mar 2021 07:23:54 GMT',
   'content-type': 'application/json',
   'content-length': '185',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3209e83d-c958-4765-879f-dd1485e0f644',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'chslsFLpiYcFRBA=',
   'x-amzn-trace-id': 'Root=1-6056f48a-6b3d6bc31faa4db95f2686f5'},
  'RetryAttempts': 0},
 'thingTypeName': 'SagemakerEdgeDemoSandBox',
 'thingTypeArn': 'arn:aws:iot:us-east-2:057716757052:thingtype/SagemakerEdgeDemoSandBox',
 'thingTypeId': 'a8d617a8-c5ab-4fad-9c42-11059a40d81f'}

In [21]:
iot_client.create_thing(
    thingName=iot_thing_name,
    thingTypeName=iot_thing_type
)

{'ResponseMetadata': {'RequestId': '8d0d334a-5f89-48ad-83f4-9adc6b7a8bb7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 21 Mar 2021 07:23:58 GMT',
   'content-type': 'application/json',
   'content-length': '187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8d0d334a-5f89-48ad-83f4-9adc6b7a8bb7',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'chsmUF1hiYcFZ1g=',
   'x-amzn-trace-id': 'Root=1-6056f48e-56a616751998cb66560d3e5a'},
  'RetryAttempts': 0},
 'thingName': 'sagemaker-edge-thing-demo-sandbox',
 'thingArn': 'arn:aws:iot:us-east-2:057716757052:thing/sagemaker-edge-thing-demo-sandbox',
 'thingId': 'b4ff8d14-468b-424c-98f8-1899a46a2b7c'}

## Device Fleet 생성

### (1) Fleet 이 사용할 Role 생성 및 정책 추가 (예: SageMaker2Fleet2gsmoon)

**[중요]: 역할 이름은 'SageMaker'로 시작해야 합니다.**
AWS 계정에서 디바이스 집합(Device Fleet)의 디바이스를 대신하여 자격 증명 공급자가 수임할 IAM 역할을 구성합니다. 



[IAM 콘솔](https://console.aws.amazon.com/iam) 로 이동하여 IoT에 대한 역할을 생성하고 다음 정책을 연결합니다.

- AmazonSageMakerEdgeDeviceFleetPolicy

### (2) 신뢰 관계에 대한 다음 문장을 추가합니다.

Add the statement to trust relationship:
```
{
  "Version": "2012-10-17",
  "Statement": [
      {
        "Effect": "Allow",
        "Principal": {"Service": "credentials.iot.amazonaws.com"},
        "Action": "sts:AssumeRole"
      },
      {
        "Effect": "Allow",
        "Principal": {"Service": "sagemaker.amazonaws.com"},
        "Action": "sts:AssumeRole"
      }
  ]
}
``
위의 role ARN을 메모하세요. device fleet 생성시 사용 됩니다.


### (3) Device Fleet 명령 실행

In [23]:
device_fleet_name ="demo2device2fleet" + str(time.time()).split('.')[0]

sagemaker_client.create_device_fleet(
    DeviceFleetName=device_fleet_name,
    RoleArn= <Role>, # arn:aws:iam::<account>:role/SageMaker* # 위에서 생성한 Role을 넣어주세요
    OutputConfig={
        'S3OutputLocation': s3_compilation_output_location
    }
)

{'ResponseMetadata': {'RequestId': '7ae751cc-b504-48bb-9c1b-a10a16c36423',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7ae751cc-b504-48bb-9c1b-a10a16c36423',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 21 Mar 2021 07:39:10 GMT'},
  'RetryAttempts': 0}}

## Device fleet에 엣지 디바이스 추가

In [24]:
device_name = "sagemaker-edge-demo-device" + str(time.time()).split('.')[0] # device name should be 36 charactors

sagemaker_client.register_devices(
    DeviceFleetName=device_fleet_name,
    Devices=[
        {          
            "DeviceName": device_name,
            "IotThingName": iot_thing_name,
            "Description": "this is a sample virtual device"
        }
    ]
)

{'ResponseMetadata': {'RequestId': '60dc387f-fb9e-4f69-ae88-ec5c6bbd511f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '60dc387f-fb9e-4f69-ae88-ec5c6bbd511f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sun, 21 Mar 2021 07:39:40 GMT'},
  'RetryAttempts': 0}}

아래는 엣지 디바이스를 위에서 추가하고, SageMaker Console의 Edge Manager의 메뉴에서 엣지 디바이스 메뉴 클릭 후에 화면 입니다. 

![DeviceRegistered.png](img/DeviceRegistered.png)

## AWS IoT에 client certificate 생성 및 등록

- 프라이빗 키, 퍼블릭 키 및 X.509 인증서 파일을 생성합니다. 
- AWS IoT에 인증서를 등록하고 활성화합니다. 
- 파일들(프라이빗 키, 퍼블릭 키 및 X.509 인증서 파일)을 저장하고 S3 버킷에 업로드 합니다.
- **이 파일들은 aws service와 통신하기 위해서 엣지 디바이스의 자격 증명을 제공하는 데 사용됩니다.**

In [25]:
iot_cert = iot_client.create_keys_and_certificate(
    setAsActive=True
)

In [26]:
with open('./iot.pem.crt', 'w') as f:
    for line in iot_cert['certificatePem'].split('\n'):
        f.write(line)
        f.write('\n')

In [27]:
with open('./iot_key.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PrivateKey'].split('\n'):
        f.write(line)
        f.write('\n')

In [28]:
with open('./iot_key_pair.pem.key', 'w') as f:
    for line in iot_cert['keyPair']['PublicKey'].split('\n'):
        f.write(line)
        f.write('\n')

- `Create_device_fleet () '에서 생성된 역할 별칭을 AWS IoT와 연결합니다.
    - 위에서 `create_device_fleet()` 실행하면 `role alias` 가 자동 생성 됩니다.

In [29]:
role_alias_name = 'SageMakerEdge-' + device_fleet_name
print("role_alias_name: ", role_alias_name)

role_alias_name:  SageMakerEdge-demo2device2fleet1616312351


In [31]:
role_alias = iot_client.describe_role_alias(
    roleAlias=role_alias_name
)
print("role_alias: ", role_alias)

role_alias:  {'ResponseMetadata': {'RequestId': '265fafeb-070e-460e-b61f-c18b70435687', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 21 Mar 2021 07:41:34 GMT', 'content-type': 'application/json', 'content-length': '377', 'connection': 'keep-alive', 'x-amzn-requestid': '265fafeb-070e-460e-b61f-c18b70435687', 'access-control-allow-origin': '*', 'x-amz-apigw-id': 'chvLSEv_CYcFogA=', 'x-amzn-trace-id': 'Root=1-6056f8ae-232103901669d66b4220793e'}, 'RetryAttempts': 0}, 'roleAliasDescription': {'roleAlias': 'SageMakerEdge-demo2device2fleet1616312351', 'roleAliasArn': 'arn:aws:iot:us-east-2:057716757052:rolealias/SageMakerEdge-demo2device2fleet1616312351', 'roleArn': 'arn:aws:iam::057716757052:role/SageMaker2Fleet2gsmoon', 'owner': '057716757052', 'credentialDurationSeconds': 3600, 'creationDate': datetime.datetime(2021, 3, 21, 7, 39, 11, 595000, tzinfo=tzlocal()), 'lastModifiedDate': datetime.datetime(2021, 3, 21, 7, 39, 11, 595000, tzinfo=tzlocal())}}


디바이스를 성공적으로 인증하기 위해 이전에 AWS IoT에 인증서를 생성하고 등록했습니다.이제 보안 토큰에 대한 요청을 인증하기 위해 정책을 만들고 인증서에 첨부해야 합니다.

In [34]:
alias_policy = {
  "Version": "2012-10-17",
  "Statement": {
    "Effect": "Allow",
    "Action": "iot:AssumeRoleWithCertificate",
    "Resource": role_alias['roleAliasDescription']['roleAliasArn']
  }
}

In [35]:
policy_name = 'aliaspolicy-'+ str(time.time()).split('.')[0]
aliaspolicy = iot_client.create_policy(
    policyName=policy_name,
    policyDocument=json.dumps(alias_policy),
)

In [36]:
iot_client.attach_policy(
    policyName=policy_name,
    target=iot_cert['certificateArn']
)

{'ResponseMetadata': {'RequestId': '6c534a14-9f3c-41eb-a126-d585dcd15c66',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 21 Mar 2021 07:43:56 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6c534a14-9f3c-41eb-a126-d585dcd15c66',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'chvhdHa4iYcF1Bg=',
   'x-amzn-trace-id': 'Root=1-6056f93c-781ad32e6d37e41b00a8ea94'},
  'RetryAttempts': 0}}

자격 증명 공급자에 대한 AWS 계정별 엔드포인트를 가져옵니다.

In [37]:
iot_endpoint = iot_client.describe_endpoint(
    endpointType='iot:CredentialProvider'
)

In [39]:
endpoint = "https://{}/role-aliases/{}/credentials".format(iot_endpoint['endpointAddress'], role_alias_name)
print("endpoint: ", endpoint)

endpoint:  https://c1oc04uiwvbz1s.credentials.iot.us-east-2.amazonaws.com/role-aliases/SageMakerEdge-demo2device2fleet1616312351/credentials


공식 Amazon 루트 CA 파일을 가져와 S3 버킷에 업로드합니다. 

In [40]:
!wget https://www.amazontrust.com/repository/AmazonRootCA1.pem

--2021-03-21 07:44:38--  https://www.amazontrust.com/repository/AmazonRootCA1.pem
Resolving www.amazontrust.com (www.amazontrust.com)... 99.86.62.116, 99.86.62.81, 99.86.62.125, ...
Connecting to www.amazontrust.com (www.amazontrust.com)|99.86.62.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1188 (1.2K) [text/plain]
Saving to: ‘AmazonRootCA1.pem’

AmazonRootCA1.pem   100%[===================>]   1.16K  --.-KB/s    in 0s      

2021-03-21 07:44:38 (118 MB/s) - ‘AmazonRootCA1.pem’ saved [1188/1188]



엔드포인트를 사용하여 보안 토큰을 반환하도록 자격 증명 공급자에게 HTTPS 요청을 보냅니다.다음 예제 명령은 curl을 사용하지만 모든 HTTP 클라이언트를 사용할 수 있습니다.

#### 자격 증명을 확인합니다.
- **[중요] 아래 주석을 해제하고 실행 해보세요. 정상적으로 에러가 없이 수행이 되어야 합니다.**
- 엔드포인트에서 오류 없이 인증서를 확인할 수 있는 경우 S3 버킷에 인증서 파일을 업로드합니다.
- **이 파일들은 EC2/장치의 [세이지메이커 에지 관리자 에이전트 설정] 섹션에서 자격 증명 공급자로 사용됩니다.**



In [54]:
# !curl --cert iot.pem.crt --key iot_key.pem.key --cacert AmazonRootCA1.pem $endpoint

In [42]:
root_ca_path = sess.upload_data('AmazonRootCA1.pem', bucket, iot_folder)
device_cert_path = sess.upload_data('iot.pem.crt', bucket, iot_folder)
device_key_path = sess.upload_data('iot_key.pem.key', bucket, iot_folder)

In [53]:
%store root_ca_path
%store device_cert_path
%store device_key_path
%store keras_model_data
%store keras_model_package
%store device_fleet_name
%store device_name
%store endpoint
%store bucket
%store iot_thing_name

Stored 'root_ca_path' (str)
Stored 'device_cert_path' (str)
Stored 'device_key_path' (str)
Stored 'keras_model_data' (str)
Stored 'keras_model_package' (str)
Stored 'device_fleet_name' (str)
Stored 'device_name' (str)
Stored 'endpoint' (str)
Stored 'bucket' (str)
Stored 'iot_thing_name' (str)
